In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag
import thor.visualize as visualize

notebook_name = "cpol_demo.ipynb"
__file__ = "/home/ewan/Documents/THOR/examples/cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [6]:
# Parent directory for saving outputs
base_local = "/home/ewan/Documents/THOR/thor"

# Create the data_options dictionary
converted_options = {
    "save": False, "load": True, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start="2005-11-13T14:00:00",
    converted_options=converted_options,
    parent_local=f"{base_local}/test/data/raw")
era5_pl_options = data.era5.data_options(parent_local=f"{base_local}/test/data/raw")
era5_sl_options = data.era5.data_options(data_format="single-levels", parent_local=f"{base_local}/test/data/raw")
data_options = option.consolidate_options([cpol_options, era5_pl_options, era5_sl_options])
dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="cpol_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
filepath = data_options["cpol"]["filepaths"][0]
with xr.open_dataset(filepath) as cpol:
    if converted_options["load"]:
        latitude, longitude = [cpol.latitude.values, cpol.longitude.values]
    else:
        lats = cpol["point_latitude"].isel(z=0).values
        lons = cpol["point_longitude"].isel(z=0).values
        latitude, longitude = grid.new_geographic_grid(lats, lons, .025, .025)
grid_options = grid.create_options(name="geographic", latitude=latitude, longitude=longitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="cpol_geographic")

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol", tags=["era5_pl", "era5_sl"])
option.save_track_options(track_options, filename="cpol_mcs")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(
        obj,
        save=True,
        style="paper",
        parent_local=f"{base_local}/test/runtime_visualization",
    )
    for obj in ["cell", "middle_cloud", "anvil"]
}
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")


2024-06-25 18:17:14,573 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/data_options/cpol_era5.yaml
2024-06-25 18:17:14,629 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/grid_options/cpol_geographic.yaml
2024-06-25 18:17:14,644 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/tag_options/era5.yaml
2024-06-25 18:17:14,647 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/track_options/cpol_mcs.yaml
2024-06-25 18:17:14,658 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/visualize_options/runtime_mcs.yaml


In [10]:
times = data.aura.generate_cpol_times(data_options["cpol"]["filepaths"])
tracks = track.simultaneous_track(times, data_options, grid_options, track_options, tag_options, visualize_options)

2024-06-25 18:23:03,358 - thor.track - INFO - Processing 2005-11-13T14:00:09.000000000.
2024-06-25 18:23:03,359 - thor.data.aura - DEBUG - Updating cpol dataset for 2005-11-13T14:00:09.000000000
2024-06-25 18:23:03,385 - thor.track - DEBUG - Processing hierarchy level 0.
2024-06-25 18:23:03,386 - thor.track - DEBUG - Tracking cell.
2024-06-25 18:23:04,881 - thor.track - DEBUG - Tracking middle_cloud.
2024-06-25 18:23:06,300 - thor.track - DEBUG - Tracking anvil.
2024-06-25 18:23:07,703 - thor.track - DEBUG - Processing hierarchy level 1.
2024-06-25 18:23:07,704 - thor.track - DEBUG - Tracking mcs.
2024-06-25 18:23:07,705 - thor.data.era5 - DEBUG - Updating era5_pl dataset for 2005-11-13T14:00:09.000000000
2024-06-25 18:23:07,707 - thor.data.era5 - DEBUG - Subsetting u_era5_oper_pl_20051101-20051130.nc
2024-06-25 18:23:07,750 - thor.data.era5 - DEBUG - Subsetting v_era5_oper_pl_20051101-20051130.nc
2024-06-25 18:23:07,793 - thor.data.era5 - DEBUG - Subsetting z_era5_oper_pl_20051101-200